In [1]:
import tensorflow as tf
from models import decoder, encoder, vae, classifier
import pandas as pd
import pathlib
from constants import *
from utils import data_loading

In [2]:
apes_info = pd.read_csv(APES_INFO_FILEPATH)
all_images_ids, train_ids, validation_ids, test_ids = data_loading.get_image_ids(apes_info, pathlib.Path(DATA_FILEPATH))

In [3]:
dataset = data_loading.load_full_dataset(DATA_FILEPATH, IMAGE_SIZE, all_images_ids)

y_train = data_loading.get_feature_dataset(apes_info, FEATURE_NAMES, "train")
y_validation = data_loading.get_feature_dataset(apes_info, FEATURE_NAMES, "validation")
y_test = data_loading.get_feature_dataset(apes_info, FEATURE_NAMES, "test")

Found 10000 files belonging to 10000 classes.


In [5]:
decoder_model = decoder.build_decoder(LATENT_DIM)
encoder_model = encoder.build_encoder(LATENT_DIM)

vae_model = vae.VAE(encoder_model, decoder_model, RECONSTRUCTION_LOSS_WEIGHT, KL_LOSS_WEIGHT)
vae_model.load_weights(MODEL_FILEPATH_VAE)

classifier_model = classifier.build_classifier(encoder_model, N_UNIQUE_FEATURES, FEATURE_NAMES)
classifier_model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(),
    loss=[tf.keras.losses.BinaryCrossentropy(from_logits=False)] * len(FEATURE_NAMES),
    metrics=tf.keras.metrics.Accuracy(),
)

In [6]:
x_train = data_loading.load_specific_dataset(dataset, train_ids, None)
train_dataset = tf.data.Dataset.zip((x_train, y_train)).batch(BATCH_SIZE)

x_validation = data_loading.load_specific_dataset(dataset, validation_ids, None)
validation_dataset = tf.data.Dataset.zip((x_validation, y_validation)).batch(BATCH_SIZE)

In [7]:
classifier_model.fit(train_dataset, validation_data=validation_dataset, epochs=2, batch_size=None)

Epoch 1/2


2023-05-23 22:24:07.311045: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_18' with dtype bool and shape [7000,23]
	 [[{{node Placeholder/_18}}]]
2023-05-23 22:24:07.311277: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype bool and shape [7000,33]
	 [[{{node Placeholder/_12}}]]


     55/Unknown - 28s 494ms/step - loss: 2.1020 - Mouth_loss: 0.2625 - Background_loss: 0.3728 - Hat_loss: 0.2904 - Eyes_loss: 0.2938 - Clothes_loss: 0.2246 - Fur_loss: 0.3069 - Earring_loss: 0.3510 - Mouth_accuracy: 0.0000e+00 - Background_accuracy: 0.0000e+00 - Hat_accuracy: 0.0000e+00 - Eyes_accuracy: 0.0000e+00 - Clothes_accuracy: 0.0000e+00 - Fur_accuracy: 0.0000e+00 - Earring_accuracy: 0.0000e+00

2023-05-23 22:24:35.805139: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_18' with dtype bool and shape [1500,23]
	 [[{{node Placeholder/_18}}]]
2023-05-23 22:24:35.805322: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_16' with dtype bool and shape [1500,37]
	 [[{{node Placeholder/_16}}]]


55/55 [==============================] - 44s 790ms/step - loss: 2.1020 - Mouth_loss: 0.2625 - Background_loss: 0.3728 - Hat_loss: 0.2904 - Eyes_loss: 0.2938 - Clothes_loss: 0.2246 - Fur_loss: 0.3069 - Earring_loss: 0.3510 - Mouth_accuracy: 0.0000e+00 - Background_accuracy: 0.0000e+00 - Hat_accuracy: 0.0000e+00 - Eyes_accuracy: 0.0000e+00 - Clothes_accuracy: 0.0000e+00 - Fur_accuracy: 0.0000e+00 - Earring_accuracy: 0.0000e+00 - val_loss: 1.2581 - val_Mouth_loss: 0.1245 - val_Background_loss: 0.2530 - val_Hat_loss: 0.1260 - val_Eyes_loss: 0.1834 - val_Clothes_loss: 0.1096 - val_Fur_loss: 0.2029 - val_Earring_loss: 0.2586 - val_Mouth_accuracy: 0.0000e+00 - val_Background_accuracy: 0.0000e+00 - val_Hat_accuracy: 0.0000e+00 - val_Eyes_accuracy: 0.0000e+00 - val_Clothes_accuracy: 0.0000e+00 - val_Fur_accuracy: 0.0000e+00 - val_Earring_accuracy: 0.0000e+00
Epoch 2/2
55/55 [==============================] - 44s 791ms/step - loss: 1.1440 - Mouth_loss: 0.1185 - Background_loss: 0.1782 - Hat_loss